In [7]:
import pytorch_lightning as pl
import torch
import torch.nn as nn
import torchvision.models as models
import torch.optim as optim
from torchvision import transforms
from sklearn.metrics import precision_recall_fscore_support, classification_report

In [2]:
import os

# go to root directory if needed
print(f"Current working directory: {os.getcwd()}")
if os.getcwd().split("/")[-1] == "notebooks":
    os.chdir("..")
    print(f"Changed working directory to: {os.getcwd()}")

Current working directory: /Users/patrickschuermann/Documents/GitHub/FlareSense/notebooks
Changed working directory to: /Users/patrickschuermann/Documents/GitHub/FlareSense


In [3]:
import src.utils.data as data


In [12]:
# Definieren Sie das PyTorch Lightning-Modell

class ResNet50BinaryClassifier(pl.LightningModule):
    def __init__(self):
        super().__init__()
        # Laden Sie das vordefinierte ResNet-50-Modell ohne den letzten Klassifikations-Layer
        self.resnet50 = models.resnet50(pretrained=False)
        self.resnet50.conv1 = nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)
        num_features = self.resnet50.fc.in_features
        self.resnet50.fc = nn.Linear(num_features, 1) 

        # Initialisieren Sie Listen, um Labels und Vorhersagen während des Tests zu sammeln
        self.test_labels = []
        self.test_preds = []
        self.val_outputs = []
        self.val_labels = []

    def forward(self, x):
        return self.resnet50(x)

    def training_step(self, batch, batch_idx):
        images, _, labels_tuple, _ = batch

        binary_labels = [0 if label == 'no_burst' else 1 for label in labels_tuple]

        binary_labels = torch.tensor(binary_labels).float().view(-1, 1)
        binary_labels = binary_labels.to(images.device)

        outputs = self(images)

        loss = nn.BCEWithLogitsLoss()(outputs, binary_labels)
        self.log('train_loss', loss)
        return loss
    
    def test_step(self, batch, batch_idx):
        images, _, labels_tuple, _ = batch

        binary_labels = [0 if label == 'no_burst' else 1 for label in labels_tuple]
        binary_labels = torch.tensor(binary_labels).float().view(-1, 1).to(images.device)

        outputs = self(images)

        loss = nn.BCEWithLogitsLoss()(outputs, binary_labels)

        probabilities = torch.sigmoid(outputs)

        predictions = (probabilities >= 0.5).int()

        # Labels und Vorhersagen für die spätere Verwendung speichern
        self.test_labels.append(binary_labels.int())
        self.test_preds.append(predictions)

        # Berechnen und protokollieren des Verlusts
        self.log('test_loss', loss, on_step=True, on_epoch=True, prog_bar=True, logger=True)

        return loss

    def on_test_epoch_end(self, outputs):
        # Konvertieren der gesammelten Daten in einzelne Tensoren
        test_labels = torch.cat(self.test_labels, dim=0)
        test_preds = torch.cat(self.test_preds, dim=0)

        # Umwandeln in Listen für sklearn
        test_labels_list = test_labels.cpu().numpy().tolist()
        test_preds_list = test_preds.cpu().numpy().tolist()

        # Berechnen von Precision, Recall und F1-Score
        precision, recall, f1, _ = precision_recall_fscore_support(test_labels_list, test_preds_list, average='binary')

        # Protokollieren der Metriken
        self.log('test_precision', precision)
        self.log('test_recall', recall)
        self.log('test_f1', f1)

        # Bereinigen der Listen für die nächste Epoche
        self.test_labels = []
        self.test_preds = []

    def validation_step(self, batch, batch_idx):
        images, _, labels_tuple, _ = batch

        binary_labels = [0 if label == 'no_burst' else 1 for label in labels_tuple]
        binary_labels = torch.tensor(binary_labels).float().view(-1, 1).to(images.device)

        outputs = self(images)
        probabilities = torch.sigmoid(outputs)
        predictions = (probabilities >= 0.5).int()

        # Labels und Vorhersagen für die spätere Verwendung speichern
        self.val_labels.append(binary_labels.int())
        self.val_preds.append(predictions)

    def on_validation_epoch_end(self):
        # Alle Validierungsdaten wurden gesammelt, und wir sind am Ende der Epoche.
        val_labels = torch.cat(self.val_labels, dim=0)
        val_preds = torch.cat(self.val_preds, dim=0)

        # Umwandeln in Listen für sklearn
        val_labels_list = val_labels.cpu().numpy().tolist()
        val_preds_list = val_preds.cpu().numpy().tolist()

        # Erstellen des Klassifikationsberichts
        report = classification_report(val_labels_list, val_preds_list, target_names=['no_burst', 'burst'])
        print("\nClassification Report:\n", report)

        # Vergessen Sie nicht, die Listen für die nächste Validierungsrunde zu leeren
        self.val_labels = []
        self.val_preds = []

    def configure_optimizers(self):
        optimizer = optim.Adam(self.parameters(), lr=0.001)
        return optimizer

# Erstellen Sie eine Instanz des Modells
model = ResNet50BinaryClassifier()

data_folder_path = "data/raw/burst_images/"

data_module = data.ECallistoDataModule(
    data_folder=data_folder_path,
    transform=transforms.Compose(
        [
            transforms.ToPILImage(),
            transforms.Resize((193, 240), antialias=True),
            transforms.ToTensor(),
        ]
    ),
    batch_size=64,
    num_workers=0,
    val_ratio=0.15,
    test_ratio=0.15,
)
data_module.setup()

# Erstellen Sie einen Trainer für das Training
trainer = pl.Trainer(max_epochs=1)

# Laden Sie den DataLoader für die Trainingsdaten
train_loader = data_module.train_dataloader()

# Starten Sie das Training
trainer.fit(model, train_loader)

test_loader = data_module.test_dataloader()
trainer.test(model, test_loader)

val_loader = data_module.val_dataloader()
trainer.validate(model, val_loader)


GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name     | Type   | Params
------------------------------------
0 | resnet50 | ResNet | 23.5 M
------------------------------------
23.5 M    Trainable params
0         Non-trainable params
23.5 M    Total params
94.015    Total estimated model params size (MB)
/Users/patrickschuermann/.local/share/virtualenvs/Deep_Learning-EdiEON_k/lib/python3.9/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:441: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=7` in the `DataLoader` to improve performance.


Epoch 0: 100%|██████████| 946/946 [29:17<00:00,  0.54it/s, v_num=24]

`Trainer.fit` stopped: `max_epochs=1` reached.


Epoch 0: 100%|██████████| 946/946 [29:17<00:00,  0.54it/s, v_num=24]


NotImplementedError: Support for `test_epoch_end` has been removed in v2.0.0. `ResNet50BinaryClassifier` implements this method. You can use the `on_test_epoch_end` hook instead. To access outputs, save them in-memory as instance attributes. You can find migration examples in https://github.com/Lightning-AI/lightning/pull/16520.